### Importar Arquivos e Bibliotecas

In [17]:
import pandas as pd
import pathlib
import win32com.client as win32

In [18]:
emails = pd.read_excel(r'Bases de Dados\Emails.xlsx')
lojas = pd.read_csv(r'Bases de Dados\Lojas.csv', encoding='latin1', sep=';')
vendas = pd.read_excel(r'Bases de Dados\Vendas.xlsx')

# display(emails)
# display(lojas)
# display(vendas)

###  Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [19]:
vendas = vendas.merge(lojas, on='ID Loja')

In [20]:

dic_lojas = {}
for loja in lojas['Loja']:
    dic_lojas[loja] = vendas.loc[vendas['Loja']==loja, :]



In [21]:
dia_indicador = vendas['Data'].max()

### Salvar a planilha na pasta de backup

In [22]:

caminho_backup = pathlib.Path(r'Backup Arquivos Lojas')

arquivos_pasta_backup = caminho_backup.iterdir()

lista_nomes_backup = [arquivo.name for arquivo in arquivos_pasta_backup]

for loja in dic_lojas:
    if loja not in lista_nomes_backup:
        nova_pasta = caminho_backup / loja
        nova_pasta.mkdir()
    
    nome_arquivo = '{}_{}_{}.xlsx'.format(dia_indicador.month, dia_indicador.day, loja)
    local_arquivo = caminho_backup / loja / nome_arquivo
    
    dic_lojas[loja].to_excel(local_arquivo)

### Calcular o indicador para 1 loja e Enviar por e-mail para o gerente

In [36]:
meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_produtos_dia = 4
meta_produtos_ano = 120
meta_ticket = 500

In [37]:
for loja in dic_lojas:

    vendas_loja = dic_lojas[loja]

    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data']==dia_indicador, :]

    # Faturamento
    faturamento_ano = vendas_loja['Valor Final'].sum()

    faturamento_dia = vendas_loja_dia['Valor Final'].sum()


    # diversidade de produtos

    qtde_produtos_ano = len(vendas_loja['Produto'].unique())
    qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())

    # Ticket Médio

    valor_venda = vendas_loja.groupby('Código Venda').sum()

    valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum()

    ticket_ano = valor_venda['Valor Final'].mean()

    ticket_dia = valor_venda_dia['Valor Final'].mean()
    
    # Enviar email
    outlook = win32.Dispatch('outlook.application')

    nome = emails.loc[emails['Loja']==loja, 'Gerente'].values[0]
    mail = outlook.CreateItem(0)
    mail.To = emails.loc[emails['Loja']==loja, 'E-mail'].values[0]
    mail.Subject = f'OnePage Dia {dia_indicador.day}/{dia_indicador.month} - Loja {lojas}'

    if faturamento_dia >= meta_faturamento_dia:
        cor_fat_dia = 'green'
    else:
        cor_fat_dia = 'red'
    if faturamento_ano >= meta_faturamento_ano:
        cor_fat_ano = 'green'
    else:
        cor_fat_ano = 'red'
    if qtde_produtos_dia >= meta_produtos_dia:
        cor_qtde_dia = 'green'
    else:
        cor_qtde_dia = 'red'
    if qtde_produtos_ano >= meta_produtos_ano:
        cor_qtde_ano = 'green'
    else:
        cor_qtde_ano = 'red'
    if ticket_dia >= meta_ticket:
        cor_ticket_dia = 'green'
    else:
        cor_ticket_dia = 'red'
    if ticket_ano >= meta_ticket:
        cor_ticket_ano = 'green'
    else:
        cor_ticket_ano = 'red'


    mail.HTMLBody = f'''
    <p>Bom dia, {nome}</p>

    <p>O resultado de ontem <strong>({dia_indicador.day}/{dia_indicador.month})</strong> da <sotrong>Loja {loja}</strong> foi:</p>

    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenário Dia</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R$ {faturamento_dia:,.2f}</td>
        <td style="text-align: center">R$ {meta_faturamento_dia:,.2f}</td>
        <td style="text-align: center"><font color="{cor_fat_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_produtos_dia}</td>
        <td style="text-align: center">{meta_produtos_dia}</td>
        <td style="text-align: center"><font color="{cor_qtde_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R$ {ticket_dia:,.2f}</td>
        <td style="text-align: center">R$ {meta_ticket:,.2f}</td>
        <td style="text-align: center"><font color="{cor_ticket_dia}">◙</font></td>
      </tr>
    </table>
    <br>
    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenário Dia</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R$ {faturamento_ano:,.2f}</td>
        <td style="text-align: center">R$ {meta_faturamento_ano:,.2f}</td>
        <td style="text-align: center"><font color="{cor_fat_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_produtos_ano}</td>
        <td style="text-align: center">{meta_produtos_ano}</td>
        <td style="text-align: center"><font color="{cor_qtde_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R$ {ticket_ano:,.2f}</td>
        <td style="text-align: center">R$ {meta_ticket:,.2f}</td>
        <td style="text-align: center"><font color="{cor_ticket_ano}">◙</font></td>
      </tr>
    </table>

    <p>Segue em anexo a planilha com todos os dados para mais detalhes.</p>

    <p>Qualquer dúvida, estou à disposição!</p>
    <p>Atenciosamente,</p>
    '''

    attachment = pathlib.Path.cwd() / caminho_backup / loja / f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'
    mail.Attachments.Add(str(attachment))

    mail.Send()
    print(f'Email da Loja {loja} enviado!')

Email da Loja Iguatemi Esplanada enviado!
Email da Loja Shopping Midway Mall enviado!
Email da Loja Norte Shopping enviado!
Email da Loja Shopping Iguatemi Fortaleza enviado!
Email da Loja Shopping União de Osasco enviado!
Email da Loja Shopping Center Interlagos enviado!
Email da Loja Rio Mar Recife enviado!
Email da Loja Salvador Shopping enviado!
Email da Loja Rio Mar Shopping Fortaleza enviado!
Email da Loja Shopping Center Leste Aricanduva enviado!
Email da Loja Ribeirão Shopping enviado!
Email da Loja Shopping Morumbi enviado!
Email da Loja Parque Dom Pedro Shopping enviado!
Email da Loja Bourbon Shopping SP enviado!
Email da Loja Palladium Shopping Curitiba enviado!
Email da Loja Passei das Águas Shopping enviado!
Email da Loja Center Shopping Uberlândia enviado!
Email da Loja Shopping Recife enviado!
Email da Loja Shopping Vila Velha enviado!
Email da Loja Shopping SP Market enviado!
Email da Loja Shopping Eldorado enviado!
Email da Loja Shopping Ibirapuera enviado!
Email da Lo

### Criar ranking para diretoria

In [43]:
faturamento_lojas_ano = vendas.groupby('Loja')[['Loja', 'Valor Final']].sum().sort_values(by='Valor Final', ascending=False)

nome_arquivo = '{}_{}_Ranking Anual.xlsx'.format(dia_indicador.month, dia_indicador.day)
faturamento_lojas_ano.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))


vendas_dia = vendas.loc[vendas['Data']==dia_indicador, :]
faturamento_lojas_dia = vendas_dia.groupby('Loja')[['Loja', 'Valor Final']].sum().sort_values(by='Valor Final', ascending=False)

nome_arquivo = '{}_{}_Ranking do Dia.xlsx'.format(dia_indicador.month, dia_indicador.day)
faturamento_lojas_dia.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))

### Enviar e-mail para diretoria

In [45]:
# Enviar email
outlook = win32.Dispatch('outlook.application')


mail = outlook.CreateItem(0)
mail.To = emails.loc[emails['Loja']=='Diretoria', 'E-mail'].values[0]
mail.Subject = f'Ranking Dia {dia_indicador.day}/{dia_indicador.month} - Loja {lojas}'

mail.Body = f'''
Prezados, bom dia

Melhor loja do DIA em faturamento: Loja {faturamento_lojas_dia.index[0]} com de faturamento R$ {faturamento_lojas_dia.iloc[0,0]:,.2f}.
Pior loja do DIA em faturamento: Loja {faturamento_lojas_dia.index[-1]} com de faturamento R$ {faturamento_lojas_dia.iloc[-1,0]:,.2f}.


Melhor loja do ANO em faturamento: Loja {faturamento_lojas_ano.index[0]} com de faturamento R$ {faturamento_lojas_ano.iloc[0,0]:,.2f}.
Pior loja do ANO em faturamento: Loja {faturamento_lojas_ano.index[-1]} com de faturamento R$ {faturamento_lojas_ano.iloc[-1,0]:,.2f}.

Segue em anexo os rankings do ano e do dia de todas as lojas.

Qualquer dúvida, estou à disposição

Atenciosamente,
Jorge Trajano
'''

attachment = pathlib.Path.cwd() / caminho_backup / f'{dia_indicador.month}_{dia_indicador.day}_Ranking Anual.xlsx'
mail.Attachments.Add(str(attachment))
attachment = pathlib.Path.cwd() / caminho_backup / f'{dia_indicador.month}_{dia_indicador.day}_Ranking do Dia.xlsx'
mail.Attachments.Add(str(attachment))

mail.Send()
print(f'Email da Diretoria enviado!')

Email da Diretoria enviado!
